# 네이버 부동산 단지정보크롤링(전매제한)
준공년월, 거래가능일 크롤링 후
거래가능시점 계산

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#from openpyxl import load_workbook
from datetime import datetime , timedelta
import numpy as np
import openpyxl

In [ ]:
#파일불러오기
down_file = ExcelFile(r'/Users/donut/PythonWork/전매제한/전매제한원장20200522.xlsx')

In [ ]:
#파일 전처리작업
df_down_file = down_file.parse(down_file.sheet_names[0])
org_col = list(df_down_file.loc[0])
df_down_file.columns = org_col
df_down_file = df_down_file.drop([df_down_file.index[0]])


conditions = [(df_down_file['total_cnt'] >= 2000),
              (df_down_file['total_cnt'] >= 1500),
              (df_down_file['total_cnt'] >= 1000),
              (df_down_file['total_cnt'] >= 700),
              (df_down_file['total_cnt'] >=300),
              (df_down_file['total_cnt'] < 300)]
    
grade = [ '2000이상','1500이상2000미만','1000이상1500미만','700이상1000미만','300이상700미만','300미만']
df_down_file['그룹'] = np.select(conditions, grade)
df_down_file['기설정여부'] = np.where(df_down_file['n1_price'] == 0,"N","Y")
df_down_file2 = df_down_file[df_down_file.build_type.isin(["아파트","아파트분양권","오피스텔","오피스텔분양권","주상복합","주상복합분양권","재개발","재건축"])]
df_down_file2 = df_down_file2.query('n1_price ==0')
#df_down_file2 = df_down_file2[~df_down_file.전매제한.isin(["제한없음"])]
df_down_file2 = df_down_file2[df_down_file.전매제한.isin(["전체전매제한", "부분전매"])]

In [ ]:
#크롤링할 단지명 리스트 : arr
arr = df_down_file2['danji_name'].values

In [ ]:
#세션처리
user_agent = ""
session = requests.Session()
session.headers.update({'user-agent' : user_agent, 'referer' : None})

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)

In [ ]:
#크롤링
arr_len = len(arr)
jungong_list = []
ganeng_list = []
jungong_ganeng_list = []
b = ['본 단지는 전매제한 단지입니다.','본 단지는 일부 면적 전매제한 단지입니다.']
c = '거래가능일'
for i in range(arr_len):
    jungong_ganeng_list = []
    driver.get("https://new.land.naver.com/")
    time.sleep(1)
    danjiname = arr[i]
    elem_search = driver.find_element_by_class_name("search_input")
    elem_search.clear()
    elem_search.send_keys(danjiname)
    elem_search.send_keys(Keys.RETURN)
    
    
    
    try :
        time.sleep(1)
        #driver.find_element_by_class_name("item_list.item_list--search")
        d1 = driver.find_elements_by_class_name("title")
        
        ddf = []
        for i2 in range(len(d1)):
            ddf.append(d1[i2].text)
            
            
        if arr[i] in ddf:
            driver.find_elements_by_class_name("title")[ddf.index(arr[i])].click()
            button = driver.find_element_by_class_name("complex_link").click()
            time.sleep(1)
            detail = driver.find_element_by_id("detailContents1")
            deep1 = detail.find_elements_by_class_name('table_th')
            deep2 = detail.find_elements_by_class_name('table_td')


            for i3 in range(len(deep1)):
                jungong_ganeng_list.append(deep1[i3].text)
                #ganeng_num_list.append(deep2[i].text)

            jungong_num =jungong_ganeng_list.index('준공년월')
            ganeng_num =jungong_ganeng_list.index('거래가능일')
            
        
            jungong_list.append(deep2[jungong_num].text)
            ganeng_list.append(deep2[ganeng_num].text)
        
        else:
            time.sleep()
            jungong_list.append("정보검색안됨")
            ganeng_list.append("정보검색안됨")
        
        
        #print("크롤링성공")
    except :
        #print("예외진입")
        try:
            d =driver.find_element_by_class_name("text_limit")

            button = driver.find_element_by_class_name("complex_link").click()
            time.sleep(1)
            detail = driver.find_element_by_id("detailContents1")
            deep1 = detail.find_elements_by_class_name('table_th')
            deep2 = detail.find_elements_by_class_name('table_td')


            for i4 in range(len(deep1)):
                jungong_ganeng_list.append(deep1[i4].text)
                #ganeng_num_list.append(deep2[i].text)

            jungong_num =jungong_ganeng_list.index('준공년월')

            if c in jungong_ganeng_list:

                ganeng_num =jungong_ganeng_list.index('거래가능일')

                jungong_list.append(deep2[jungong_num].text)
                ganeng_list.append(deep2[ganeng_num].text)
            else:
                jungong_list.append(deep2[jungong_num].text)
                ganeng_list.append("거래가능일조회안됨")
        except:
            jungong_list.append("전매 or 부분전매 아님")
            ganeng_list.append("전매 or 부분전매 아님")
    
    #time.sleep(1)
    cname= arr[i]
    #nnum = len(arr)-arr.index(arr[i])-1

    nnum = len(arr)-np.where(arr[i] == arr)[0][0]-1

    print('{} 크롤링성공, {}개 남음'.format(cname,nnum))

In [ ]:
#준공일자, 거래가능일자 리스트 가공(데이터형식)
jungong_list2 = []
ganeng_list2 = []

for i in range(len(jungong_list)):
    if jungong_list[i][:2] !='20':
        jungong_list2.append('')
    else:
        a = jungong_list[i][:4]
        b = jungong_list[i][6:8]
        c = str(a+'-'+b+'-01')
        jungong_list2.append(c)
        
        
for i in range(len(ganeng_list)):
    if ganeng_list[i][:2] !='20':
        ganeng_list2.append('')
    else:
        a = ganeng_list[i][:4]
        b = ganeng_list[i][6:9]
        b2 = b.replace("월","")
        b3 = b2.replace(" ","")
        if int(b3) < 10:
            b3 = '0'+b3
        c = ganeng_list[i][9:11]
        c2 = c.replace(" ","")
        c3 = c2.replace("일","")
        if int(c3) <10:
            c3 = '0' +c3
        elif int(c3) == 31:
            c3 = '30'
        d = str(a+'-'+b3+'-'+c3)
        ganeng_list2.append(d)

In [ ]:
#불러왔던 파일에 리스트 첨부하기
df_down_file3 = df_down_file2.copy()
df_down_file3['입주일자'] = jungong_list2
df_down_file3['전매가능일자'] = ganeng_list2
df_down_file4 = df_down_file3.reset_index(drop=True)

In [ ]:
#데이터 후처리(입주일계산)
now = datetime.now()
timecal1 =[]
for i in range(df_down_file4.shape[0]):
    if df_down_file4['입주일자'][i] == '':
        timecal1.append(0)
    else:
        time1 = df_down_file4['입주일자'][i]
        time2 = datetime.strptime(time1 ,'%Y-%m-%d')
        time3 = datetime.now()
        #time3 = now.strftime('%Y-%m-%d')
        timecal1.append((time2-time3).days)
        
df_down_file4['입주일구분'] = timecal1

conditions = [  (df_down_file4['입주일구분'] > 0),
                (df_down_file4['입주일구분'] == 0),
                (df_down_file4['입주일구분'] < 0)]
grade = [ '입주전','','초과']


df_down_file4['입주일 비교'] = np.select(conditions, grade)


In [ ]:
#전매가능일 계산
timecal2 =[]
for i in range(df_down_file4.shape[0]):
    if df_down_file4['전매가능일자'][i] == '':
        timecal2.append(0)
    else:
        time1 = df_down_file4['전매가능일자'][i]
        time2 = datetime.strptime(time1 ,'%Y-%m-%d')
        time3 = datetime.now()
        #time3 = now.strftime('%Y-%m-%d')
        timecal2.append((time2-time3).days)
    #print(df_down_file4['전매가능일자'][i] + "성공")


df_down_file4['전매가능일구분'] = timecal2

conditions = [  (df_down_file4['전매가능일구분'] > 0),   
                (df_down_file4['전매가능일구분'] == 0),
                (df_down_file4['전매가능일구분'] < 0)]
grade = [ '해제전','','전매해제']


df_down_file4['전매가능일 비교'] = np.select(conditions, grade)

In [ ]:
#매매가능일계산
timecal3 = []
for i in range(df_down_file4.shape[0]):
    if df_down_file4['전매가능일자'][i] == '':
        timecal3.append('')
    else:
        time1 = df_down_file4['전매가능일자'][i]
        time2 = datetime.strptime(time1,'%Y-%m-%d')
        time3 = time2 - timedelta(days=90)
        timecal3.append(time3)
df_down_file4['매매가능일'] = timecal3
df_down_file4['매매가능일'].fillna(value = '', inplace = True)

In [ ]:
#전월세가능일계산
timecal4 = []
for i in range(df_down_file4.shape[0]): 
    if df_down_file4['입주일 비교'][i] == '초과':
        timecal4.append("제한없음")
    elif df_down_file4['입주일 비교'][i] == '':
        timecal4.append('')
    else:
        time1 = df_down_file4['입주일자'][i]
        time2 = datetime.strptime(time1,'%Y-%m-%d')
        time3 = time2 - timedelta(days=90)
        timecal4.append(time3)
df_down_file4['전월세가능일'] = timecal4
df_down_file4['전월세가능일'].fillna(value = '', inplace = True)
    

In [ ]:
#거래가능시점계산
timecal5 =[]
for i in range(df_down_file4.shape[0]):
    if df_down_file4['매매가능일'][i] == '':
        timecal5.append(0)
        
    elif df_down_file4['전월세가능일'][i] =='':
        timecal5.append(0)
    
    elif df_down_file4['전월세가능일'][i] =='제한없음':
        timecal5.append(df_down_file4['매매가능일'][i])

    else:
        time1 = df_down_file4['매매가능일'][i]        
        time2 = df_down_file4['전월세가능일'][i]
        time3 = (time1 - time2).days
        if time3 > 0:
            timecal5.append(df_down_file4['전월세가능일'][i])
        else:
            
            timecal5.append(df_down_file4['매매가능일'][i])

df_down_file4['가능시점'] = timecal5



In [ ]:
#매매가능시점계산
timecal6 = []
for i in range(df_down_file4.shape[0]):
    if df_down_file4['전매제한'][i] == '전체전매제한':
        if df_down_file4['매매가능일'][i] =='':
            timecal6.append('')
        else:    
            time1 = df_down_file4['매매가능일'][i]
            time2 = datetime.now()
            time3 = (time1-time2).days
            if time3 > 0:
                timecal6.append('불가')
            else:
                timecal6.append('가능')

    elif df_down_file4['전매제한'][i] == '부분전매':
        timecal6.append('가능')

df_down_file4['매매 시점'] = timecal6



In [ ]:
#전월세가능시점계산1
timecal7= []
for i in range(df_down_file4.shape[0]):
    if df_down_file4['전매제한'][i] == '전체전매제한':
        if df_down_file4['입주일 비교'][i] == '입주전':
            time1 = df_down_file4['전월세가능일'][i]
            time2 = datetime.now()
            time3 = (time1-time2).days
            if time3 > 0:
                timecal7.append('불가')
            else:
                timecal7.append('가능')
        elif df_down_file4['입주일 비교'][i] == '':
            timecal7.append('')
        else:
            timecal7.append('가능')
    elif df_down_file4['전매제한'][i] == '부분전매':
        timecal7.append('가능')
        
df_down_file4['전월세 시점'] = timecal7

In [ ]:
#전월세가능시점계산2
timecal8 = []
for i in range(df_down_file4.shape[0]):
    if df_down_file4['매매 시점'][i] == '불가':
        if df_down_file4['전월세 시점'][i] =='불가':
            timecal8.append('불가')
        else:
            timecal8.append('가능')
    elif df_down_file4['매매 시점'][i]=='':
        timecal8.append('')
    else:
        timecal8.append('가능')
df_down_file4['가능여부'] = timecal8

In [ ]:
#엑셀파일저장
writer = ExcelWriter('%s-%s-%s 전매제한.xlsx'%(now.year, now.month, now.day))
df_down_file4.to_excel(writer, '최종리스트')


wb=openpyxl.load_workbook('./%s-%s-%s 전매제한.xlsx'%(now.year, now.month, now.day))
sheet1 =wb['최종리스트']


sheet1['A1'] ='단지코드'


wb.save('%s-%s-%s 전매제한.xlsx'%(now.year, now.month, now.day))
#writer.save()
